In [1]:
!wget https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt

--2024-12-26 18:53:45--  https://gist.githubusercontent.com/phillipj/4944029/raw/75ba2243dd5ec2875f629bf5d79f6c1e4b5a8b46/alice_in_wonderland.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148574 (145K) [text/plain]
Saving to: ‘alice_in_wonderland.txt’

alice_in_wonderland 100%[===================>] 145.09K  --.-KB/s    in 0.03s   

2024-12-26 18:53:45 (4.53 MB/s) - ‘alice_in_wonderland.txt’ saved [148574/148574]



In [2]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [4]:
with open("alice_in_wonderland.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [5]:
sentences = [simple_preprocess(sentence) for sentence in text.splitlines() if sentence]

In [6]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

In [7]:
model.wv.most_similar("alice", topn=5)

[('to', 0.9997630715370178),
 ('this', 0.9997443556785583),
 ('and', 0.9997442364692688),
 ('it', 0.9997388124465942),
 ('in', 0.9997347593307495)]

In [8]:
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
sentences = [
    [word for word in simple_preprocess(sentence) if word not in stop_words]
    for sentence in text.splitlines() if sentence
]

In [10]:
model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)

In [11]:
model.wv.most_similar("alice", topn=5)

[('said', 0.9758531451225281),
 ('little', 0.9634153842926025),
 ('one', 0.9624069333076477),
 ('time', 0.9573027491569519),
 ('went', 0.9491702318191528)]

In [13]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [14]:
dictionary = Dictionary(sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences]

In [15]:
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

In [16]:
topics = lda_model.print_topics(num_words=5)
for idx, topic in topics:
    print(f"Topic {idx}: {topic}")

Topic 0: 0.020*"one" + 0.016*"queen" + 0.015*"little" + 0.014*"alice" + 0.012*"two"
Topic 1: 0.030*"alice" + 0.017*"went" + 0.016*"rabbit" + 0.015*"little" + 0.014*"jury"
Topic 2: 0.017*"gryphon" + 0.012*"soup" + 0.011*"next" + 0.011*"beautiful" + 0.011*"hatter"
Topic 3: 0.030*"said" + 0.025*"would" + 0.024*"alice" + 0.017*"one" + 0.013*"could"
Topic 4: 0.112*"said" + 0.048*"alice" + 0.028*"turtle" + 0.027*"mock" + 0.024*"king"


In [17]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.9 MB/s eta 0:00:00


In [18]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

lda_display = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

pyLDAvis.display(lda_display)

In [19]:
pyLDAvis.save_html(lda_display, 'lda_visualization.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
